In [9]:
import numpy as np
import random

In [10]:
def calculate_remainder(message, generator):
    remainder = list(message)
    while len(remainder) >= len(generator):
        for index in range(len(generator)):
            remainder[-len(generator) + index] ^= generator[index]
        while remainder and remainder[-1] == 0:
            remainder.pop()
    return np.array(remainder)

def multiply_polynomials(poly1, poly2):
    product = np.zeros(len(poly1) + len(poly2) - 1, dtype=int)
    for idx, bit in enumerate(poly2):
        if bit:
            product[idx:idx + len(poly1)] = np.bitwise_xor(product[idx:idx + len(poly1)], poly1.astype(int))
            
    return product

def introduce_and_correct_errors(original, generator_poly, num_errors):
    transmitted = multiply_polynomials(original, generator_poly)
    print(f"Отправленное сообщение: {transmitted}")
    
    received = transmitted.copy()
    error_vector = np.zeros(received.size, dtype=int)
    error_positions = random.sample(range(received.size), num_errors)
    for pos in error_positions:
        error_vector[pos] = 1
    received = (received + error_vector) % 2
    print(f"Сообщение с ошибками: {received}")
    
    remainder = calculate_remainder(received, generator_poly)
    error_patterns = [[1]] if num_errors == 1 else [[1, 1, 1], [1, 0, 1], [1, 1], [1]]
    
    shift_count = 0
    while not any(np.array_equal(remainder, pattern) for pattern in error_patterns):
        shifted = multiply_polynomials(remainder, np.array([0, 1]))
        remainder = calculate_remainder(shifted, generator_poly)
        shift_count += 1
    
    correction = np.zeros(received.size, dtype=int)
    correction[-shift_count - 1 if shift_count else 0] = 1
    corrected = (received + multiply_polynomials(remainder, correction)[:received.size]) % 2
    print(f"Исправленное сообщение: {corrected}\n")

6.1 Написать функции кодирования и декодирования для циклического кода (7,4) с порождающим многочленом 𝑔(𝑥) = 1 + 𝑥2 + 𝑥, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.

In [11]:
# Определение сообщений и порождающих полиномов
messages = [
    np.array([1, 0, 0, 1]),
    np.array([1, 0, 0, 1, 0, 0, 0, 1, 1])
]
generators = [
    np.array([1, 0, 1, 1]),
    np.array([1, 0, 0, 1, 1, 1, 1])
]



poly_description = 'g = 1 + x^2 + x^3' 
print(f"{poly_description}")
print(f"Исходное сообщение: {messages[0]}")
print(f"Порождающий полином: {generators[0]}\n")
    
max_errors = 3
for error_count in range(1, max_errors + 1):
    introduce_and_correct_errors(messages[0], generators[0], error_count)


g = 1 + x^2 + x^3
Исходное сообщение: [1 0 0 1]
Порождающий полином: [1 0 1 1]

Отправленное сообщение: [1 0 1 0 0 1 1]
Сообщение с ошибками: [0 0 1 0 0 1 1]
Исправленное сообщение: [1 0 1 0 0 1 1]

Отправленное сообщение: [1 0 1 0 0 1 1]
Сообщение с ошибками: [1 1 1 0 0 1 0]
Исправленное сообщение: [1 1 1 0 0 0 0]

Отправленное сообщение: [1 0 1 0 0 1 1]
Сообщение с ошибками: [1 1 1 1 0 1 0]
Исправленное сообщение: [0 1 1 1 0 1 0]


Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом 𝑔(𝑥) = 1 + 𝑥3 + 𝑥4 + 𝑥5 + 𝑥6, исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4. Обратите внимание, что пакет ошибок длины t не означает, что все разряды в пределах этого пакета изменятся (см. лекции).

In [12]:
poly_description = 'g = 1 + x^3 + x^4 + x^5 + x^6' 
print(f"{poly_description}")
print(f"Исходное сообщение: {messages[1]}")
print(f"Порождающий полином: {generators[1]}\n")
    
max_errors = 4
for error_count in range(1, max_errors + 1):
    introduce_and_correct_errors(messages[1], generators[1], error_count)

g = 1 + x^3 + x^4 + x^5 + x^6
Исходное сообщение: [1 0 0 1 0 0 0 1 1]
Порождающий полином: [1 0 0 1 1 1 1]

Отправленное сообщение: [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибками: [1 0 0 0 1 0 0 0 0 1 1 0 0 0 1]
Исправленное сообщение: [1 0 0 0 0 0 0 0 0 1 1 0 0 0 1]

Отправленное сообщение: [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибками: [1 0 0 0 1 1 0 1 0 1 0 0 0 0 1]
Исправленное сообщение: [1 0 0 0 1 1 0 1 0 1 1 1 1 0 1]

Отправленное сообщение: [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибками: [1 1 0 0 0 1 0 0 0 1 0 0 0 0 1]
Исправленное сообщение: [1 1 0 0 0 1 0 0 0 1 0 1 0 1 1]

Отправленное сообщение: [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибками: [1 0 0 0 1 0 1 1 0 1 1 0 0 0 0]
Исправленное сообщение: [1 0 0 0 1 0 1 1 0 0 1 1 0 0 0]
